I should note that you will need PhoREAL for this. It was far more painful to try and include everything myself.

In [1]:
import sys

sys.path.insert(1,'/directory/path/to/PhoREAL')

from phoreal.reader import get_atl03_struct

In [2]:
atl03path = "processed_ATL03_20210804143024_06411202_006_01.h5"
atl08path = "processed_ATL08_20210804143024_06411202_006_01.h5"

In [3]:
atl03 = get_atl03_struct(atl03path, 'gt1r', atl08path)

You can see below that indices 19 and 20 both correspond to time.

In [4]:
atl03.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7168 entries, 0 to 7167
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   delta_time      7168 non-null   float64
 1   dist_ph_across  7168 non-null   float32
 2   dist_ph_along   7168 non-null   float64
 3   h_ph            7168 non-null   float32
 4   lat_ph          7168 non-null   float64
 5   lon_ph          7168 non-null   float64
 6   pce_mframe_cnt  7168 non-null   uint32 
 7   ph_id_channel   7168 non-null   uint8  
 8   ph_id_count     7168 non-null   uint8  
 9   ph_id_pulse     7168 non-null   uint8  
 10  quality_ph      7168 non-null   int8   
 11  signal_conf_ph  7168 non-null   int8   
 12  weight_ph       7168 non-null   uint8  
 13  classification  7168 non-null   int64  
 14  norm_h          7166 non-null   float64
 15  seg_id          7168 non-null   int32  
 16  ph_bihr         7168 non-null   float64
 17  ph_bcr          7168 non-null   f

I'll try to call the time column from the DataFrame.

In [5]:
atl03.df['time']

,time,time
0,0.0000,0.0000
1,0.0000,0.0000
2,0.0000,0.0000
3,0.0001,0.0001
4,0.0001,0.0001
...,...,...
7163,0.1801,0.1801
7164,0.1801,0.1801
7165,0.1801,0.1801
7166,0.1801,0.1801


We have two columns of the exact same thing. We can look at why, I've put arrows at the relevant lines.

In [ ]:
#This is the function as defined in the reader.py script

def get_atl03_struct(atl03filepath, gt, atl08filepath = None, epsg = None, 
                     kml_bounds_txt = None, header_file_path = None):
    df = read_atl03_heights_data(atl03filepath, gt)
    if atl08filepath:
        try:
            # df = merge_label_to_df(atl03filepath, atl08filepath, gt, df)
            df['classification'] = get_atl03_classification(atl03filepath,\
                                                            atl08filepath, gt)
            # df = merge_norm_h_to_df(atl03filepath, atl08filepath, gt, df)
            df['norm_h'] = get_atl03_heights_offset(atl03filepath,\
                                                    atl08filepath, gt)
            dataIsMapped = True
        except:
            dataIsMapped = False
    else:
        dataIsMapped = False
    df['seg_id'] =  get_atl03_segment_id(atl03filepath, gt)
    df['ph_bihr'], df['ph_bcr'], df['ph_rate'] =\
        get_atl03_rate(atl03filepath, gt) 
    df['dist_ph_along'] = get_atl03_dist_ph_along(atl03filepath, gt)
    df['time'] = df['delta_time'] -\                                         # <-- first assignment of time
        np.min(np.array(df.delta_time)[np.nonzero(np.array(df.delta_time))]) 
    df = get_atl_time(df)                                                    # <-- second assignment of time
    df, epsg = get_atl_coords(df, epsg)
    df, rotation_data = get_atl_alongtrack(df)
    track_direction = get_direction(np.array(df.lat_ph))
    atl03filename = get_file_name(atl03filepath)
    atl03_info = getNameParts(atl03filename)
    hemi, zone = identify_hemi_zone(epsg)

    if dataIsMapped:
        # for some reason, need to reset nan to -1,
        # evemn though it's done in get_atl03_classification..
        c = np.array(df.classification)
        nan_index = np.where(np.isnan(c))
        c[nan_index] = -1 # assign nan to -1
        c = c.astype(int)
        df.classification = c
        # df.replace({'classification' : np.nan}, -1)

    # Assign everything to the struct
    beamNum = GtToBeamNum(atl03filepath, gt)
    beamStrength = GtToBeamSW(atl03filepath, gt)
    atl03Struct = AtlStruct(df, gt, beamNum, beamStrength, epsg, zone, hemi,
                              atl03filepath, atl03filename, track_direction,
                              'ATL03', atl03_info, dataIsMapped,rotation_data)

        
    return atl03Struct

What does get_at_time do?

In [ ]:
#Calculate alongtrack time
def get_atl_time(df):
    delta_time = np.array(df['delta_time'])
    min_detla_time = np.min(delta_time[np.nonzero(delta_time)])
    time = delta_time - min_detla_time
    
    df = pd.concat([df,pd.DataFrame(time,
                                    columns=['time'])],axis=1)
    return df

This is the exact same thing as the first assignment. We can get rid of one of the assignments, preferably the first one because the code will be neater.